# Simulators

Sometimes when we assemble algorithms, it is useful to check whether or not the resulting state is consistent with our expectation. As the state is not something we have access to with experiment, we need to use a simulator to obtain the exact final output state. In QURI Parts, a simulator is any function that work with the explicit state vector or density matrices.

## Qulacs simulators

In the `quri_parts.qulacs.simulator` module, we provide 3 simulator features for state vectors.

- `evaluate_state_to_vector`
- `run_circuit`
- `get_marginal_probability`

We introduce their functionalities in the following sections.

### `evaluate_state_to_vector`

`evaluate_state_to_vector` is a function that converts any non-parametric states into a `QuantumStateVector` with an empty circuit.

In [ ]:
from quri_parts.qulacs import evaluate_state_to_vector # >= 0.22.0
from quri_parts.core import quantum_state # >= 0.22.0

#### Converts a `ComputationalBasisState`
For example, `bits = 2 = 0b10` represents $|10\rangle$. Note that vectors are arranged as follows: |00>, |01>, |10>, |11>, ... .

In [ ]:
comp_state = quantum_state(n_qubits=2, bits=2)

out_state = evaluate_state_to_vector(comp_state)

print("State vector:")
print(out_state.vector)
print("")
print("Circuit:")
print(out_state.circuit.gates)

State vector:
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]

Circuit:
()


#### Converts a `GeneralCircuitQuantumState`

In [ ]:
from quri_parts.circuit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.add_H_gate(0)
circuit.add_CNOT_gate(0, 1)

bell_state = quantum_state(n_qubits=2, circuit=circuit)
out_state = evaluate_state_to_vector(bell_state)

print("State vector:")
print(out_state.vector)
print("")
print("Circuit:")
print(out_state.circuit.gates)

State vector:
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]

Circuit:
()


#### Converts a `QuantumStateVector`

In [ ]:
import numpy as np
from scipy.stats import unitary_group


circuit = QuantumCircuit(2)
circuit.add_H_gate(0)
circuit.add_CNOT_gate(0, 1)

init_state_vector = unitary_group.rvs(4)[:, 0]
state = quantum_state(n_qubits=2, vector=init_state_vector, circuit=circuit)
out_state = evaluate_state_to_vector(state)

print("State vector:")
print(out_state.vector)
print("")
print("Circuit:")
print(out_state.circuit.gates)

State vector:
[ 0.32593894-0.27028198j -0.42802279-0.35593234j -0.43789377-0.54368153j
 -0.05890213-0.14145707j]

Circuit:
()


### `run_circuit`

`run_circuit` is a function that acts a quantum circuit on a state vector represented by a `numpy.array` and returns the resulting state vector.

In [ ]:
from quri_parts.qulacs.simulator import run_circuit

n_qubits = 2

circuit = QuantumCircuit(n_qubits)
circuit.add_H_gate(0)
circuit.add_CNOT_gate(0, 1)

init_state_vector = unitary_group.rvs(2**n_qubits)[:, 0]
out_state = run_circuit(circuit, init_state_vector)
out_state

array([-0.09463788-0.52276306j,  0.33038746-0.7447442j ,
        0.20809496-0.08257501j,  0.03027456+0.05407367j])

### `get_marginal_probability`

Sometimes we need to perform partial measurement to a state. `get_marginal_probability` returns the probability of obtaining the specified computational basis eigenstate after measuring a subset of the qubits. For example, for a state with 3 qubits, setting `measured_values={0: 1, 2: 0}` outputs the probability of obtaining 1 from the 0th qubit and 0 from the 2nd qubit. That is, the probability is obtained from the coefficients of $|001\rangle$ and $|011\rangle$.

In [ ]:
from quri_parts.qulacs.simulator import get_marginal_probability

n_qubits = 3

init_state = unitary_group.rvs(2**n_qubits)[:, 0]

state = quantum_state(n_qubits=n_qubits, vector=init_state)
out_state = evaluate_state_to_vector(state)
print("State vector:")
print(out_state.vector)
print("")
print(
    "Probability of measuring 1 from to 0th qubit and 0 from the 2nd qubit:",
    get_marginal_probability(init_state, measured_values={0: 1, 2: 0})
)

State vector:
[ 0.34533374+0.22062691j  0.09821408-0.18055882j  0.08551216-0.48568241j
  0.02946622+0.3602851j  -0.27363407-0.31395563j -0.02803822-0.19829083j
 -0.40354924+0.10441679j  0.16587578-0.0335971j ]

Probability of measuring 1 from to 0th qubit and 0 from the 2nd qubit: 0.1729211069889257


## Stim simulators

We also provide

- `evaluate_state_to_vector`
- `run_circuit`

in the `quri_parts.stim.simulator` module. The functionalities are the same as those in `quri_parts.qulacs.simulator` except that they only accept Clifford circuits and performs the circuit execution using [Stim](https://github.com/quantumlib/Stim).